### Adaptive RAG

### Importing necessary libraries 

In [1]:
import torch
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma


In [2]:
print(torch.cuda.is_available())
print("*"*100)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using torch {torch.__version__} ({DEVICE})")

True
****************************************************************************************************
Using torch 2.1.2+cu118 (cuda)


In [3]:
model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

d:\GPT\New approach\20October_restart_working\satnam\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\GPT\New approach\20October_restart_working\satnam\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
loader = DirectoryLoader('Data',
                        glob='*.pdf',
                        loader_cls=PyPDFLoader)

documents = loader.load()

In [5]:
len(documents)

224

In [6]:
unique_sources = set()

for doc in documents:
    if 'source' in doc.metadata:
        unique_sources.add(doc.metadata['source'])

unique_sources = list(unique_sources)

In [7]:
print("Number of unique sources are : " , len(unique_sources))
print("Unique sources:", unique_sources)

Number of unique sources are :  9
Unique sources: ['Data\\lebs1ps.pdf', 'Data\\lebs107.pdf', 'Data\\lebs101.pdf', 'Data\\lebs103.pdf', 'Data\\lebs102.pdf', 'Data\\lebs105.pdf', 'Data\\lebs104.pdf', 'Data\\lebs106.pdf', 'Data\\lebs108.pdf']


In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents=documents)

vector_store = Chroma.from_documents(texts, embeddings, collection_metadata={"hnsw:space": "cosine"}, persist_directory="stores/data_cosine")

print("*"*100)
print("Chroma Vectore Store Created: " , vector_store)
print("*"*100)

****************************************************************************************************
Chroma Vectore Store Created:  <langchain_community.vectorstores.chroma.Chroma object at 0x0000018DD3A3F3A0>
****************************************************************************************************


In [9]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [10]:
retriever.invoke("principles of management")

[Document(metadata={'page': 34, 'source': 'Data\\lebs102.pdf'}, page_content='Principles of management are general guidelines, which can be used for conduct in work places under certain situations. They help managers to take and implement decisions.NatureThe nature of management principles can be discussed under the heads- formed by practice; general guidelines; universal; flexible; behavioural; contingent; and cause and effect relationship SignificanceProper understanding of significance of management principles is essential to make sound decisions by managers. The significance can be discussed under the following heads- Increase in efficiency; Optimum utilisation of resources; Scientific decision making; Adaptation to changing environment; Fulfilling social responsibilities; Proper research and development; Training managers; and Effective administration.Scientific ManagementTaylor’s principles of scientific management are — Science, not the rule of thumb; Harmony not discord; Cooper

In [11]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model='mistral', format="json", temperature=0)

In [16]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

#### Query Route

In [12]:

prompt = PromptTemplate(
    template="""You are an expert at routing a user question to a vectorstore or web search. \n
    Use the vectorstore for questions on Business Studies, Principles of Management and Business Environment. \n
    You do not need to be stringent with the keywords in the question related to these topics. \n
    Otherwise, use web-search. Give a binary choice 'web_search' or 'vectorstore' based on the question. \n
    Return the a JSON with a single key 'datasource' and no premable or explanation. \n
    Question to route: {question}""",
    input_variables=["question"],
)

question_router = prompt | llm | JsonOutputParser()


In [13]:
question = "mantain discipline in an organization"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(question_router.invoke({"question": question}))
print("#"*100)
question = "how to apply for a new job?"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(question_router.invoke({"question": question}))

d:\GPT\New approach\20October_restart_working\satnam\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'datasource': 'vectorstore'}
####################################################################################################
{'datasource': 'web_search'}


#### Grading our retrieval

In [14]:

prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()


In [15]:
question = "mantain discipline in an organization"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))
print("#"*100)
question = "how to apply for a new job?"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}
####################################################################################################
{'score': 'no'}


#### Generation

In [17]:

# Prompt
prompt = PromptTemplate(
    template="""Use the following pieces of document to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Document: {document}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:

""",

    input_variables=["question", "document"],

)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run

question = "process of recruitmment?"
generation = rag_chain.invoke({"document": docs, "question": question})
print(generation)



{ "process": ["writing job description and candidate profile", "generating information for 'situations vacant' advertisement", "locating potential candidates or determining sources of potential candidates"] }


#### Hallucination Grader

In [18]:

prompt = PromptTemplate(
    template="""You are a grader assessing whether an answer is grounded in / supported by a set of facts. \n 
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}
    Give a binary score 'yes' or 'no' score to indicate whether the answer is grounded in / supported by a set of facts. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'yes'}

#### Answer Grader

In [19]:

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing whether an answer is useful to resolve a question. \n 
    Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question}
    Give a binary score 'yes' or 'no' to indicate whether the answer is useful to resolve a question. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})

{'score': 'yes'}

#### Question Re-writer

In [21]:


re_write_prompt = PromptTemplate(
    template="""
    You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the initial and formulate an improved question. \n
     Here is the initial question: \n {question}. 
     Improved question with no preamble: \n 
     """,
    input_variables=["generation", "question"],
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'{"recruitment process": "What is the sequence of steps involved in recruiting new employees?"}\n\n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    '

: 